In [10]:
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

input = pd.read_excel("input_score_last.xlsx")   

y_feature = input['Score']
X_feature = input.drop('Score',axis=1)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_feature, y_feature, test_size=0.2, 
random_state =50)



#for import machine learning Algorithm
# from sklearn.linear_model import LinearRegression 
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.ensemble import GradientBoostingRegressor
# from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_val_score

def get_model_cv_prediction(model, X_data, y_target):
    neg_mse_scores = cross_val_score(model, X_data, y_target, scoring = "neg_mean_squared_error",cv=5)
    rmse_scores = np.sqrt(-1*neg_mse_scores)
    avg_rmse = np.mean(rmse_scores)
    
    print('#### ',model.__class__.__name__, ' ####')
    print(' 5 교차 검증의 평균 RMSE : {0:.3f}'.format(avg_rmse))

# lr_reg = LinearRegression(normalize=True) #True를 False 넣어서도 봐주세요
# dt_reg = DecisionTreeRegressor(random_state=0, min_samples_split=6,min_samples_leaf=5, max_depth=5) 
# rf_reg = RandomForestRegressor(random_state=0, n_estimators = 200,max_depth=4, min_samples_leaf=7)
# gb_reg = GradientBoostingRegressor(random_state=0, n_estimators = 20)
# xgb_reg = XGBRegressor(random_state=0, objective='reg:squarederror', n_estimators=500,learning_rate=0.01)
lgb_reg = LGBMRegressor(random_state=0, n_estimators=20)

# models = [lr_reg, dt_reg, rf_reg, gb_reg, xgb_reg, lgb_reg]
models = [lgb_reg]
for model in models:
    get_model_cv_prediction(model, X_feature, y_feature)

#fit == I will training use X_train & Y_train and lgb_reg
lgb_reg.fit(X_train, y_train)
#y_preds == predict result using X_test
y_preds = lgb_reg.predict(X_test)

mse = mean_squared_error(y_test,y_preds)
rmse = np.sqrt(mse)

print('\n####After Test####')
print('MSE : {0:.3f}, RMSE : {1:.3f}'.format(mse,rmse))
print('Variance score(R2score) : {0:.3f}'.format(r2_score(y_test, y_preds)))

####  LGBMRegressor  ####
 5 교차 검증의 평균 RMSE : 0.975

####After Test####
MSE : 0.940, RMSE : 0.970
Variance score : 0.966
